In [7]:
%reload_ext autoreload
%autoreload 2

In [23]:
import numpy as np
import torch_geometric as tg
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [16]:
# Local imports
from l2gv2.datasets import get_dataset
from l2gv2.graphs import TGraph, induced_subgraph
from l2gv2.patch import create_patch_data
from l2gv2.patch.clustering import louvain_clustering, metis_clustering
from l2gv2.embedding import patch_embeddings
from l2gv2.align.utils import preprocess_graphs, get_embedding, intersections_nodes
from l2gv2.align.geo import train_model


ModuleNotFoundError: No module named 'l2gv2.embedding.layers'

# <font color="grey"> Local2Global Alignment</font>

We use the Cora citation index dataset for the experiments.

In [10]:
cora = get_dataset("Cora")
cora = TGraph(edge_index=cora[0].edge_index, 
              edge_attr=cora[0].edge_attr,  
              num_nodes=cora[0].num_nodes, 
              ensure_sorted=True, 
              undir=False)

Loading edge and node data from memory


In [11]:
min_overlap = 100
target_overlap = 200
dim = 64

In [12]:
partition_tensor = louvain_clustering(cora)

In [13]:
pt, pgraph= create_patch_data(cora, 
                              partition_tensor=partition_tensor,
                              min_overlap=min_overlap, 
                              target_overlap=target_overlap, 
                              verbose=True)

number of patches: 11
average patch degree: 3.909090909090909


enlarging patch overlaps:   0%|          | 0/11 [00:00<?, ?it/s]

In [18]:
patches = [induced_subgraph(cora, p) for p in pt]
neg_edges = tg.utils.negative_sampling(cora.edge_index, num_nodes=cora.num_nodes)

In [ ]:
patches_emb, _ = patch_embeddings(patches, dim=dim, num_epochs=200, device=device)
nodes = intersections_nodes(patches)
n_patches=len(patches)

In [ ]:

emb_patches = preprocess_graphs(patches_emb, nodes)
res, loss_hist = train_model(emb_patches, dim, n_patches , num_epochs=200, learning_rate=0.05, verbose=False)
emb = get_embedding(patches_emb, res)
full_model_ip = tg.nn.VGAE(encoder=VGAEconv(dim, test_data.num_node_features))
auc, ap = full_model_ip.test(torch.tensor(emb), test_data.edge_index, neg_edges)